In [ ]:
import cv2
from keras.models import load_model
import numpy as np
import os

model = load_model('emojinator.h5')

def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
4 0.99971765
6 0.93823195
6 0.66991496
5 0.44145116
6 0.9024679
6 0.7850788
6 0.46603894
6 0.767629
6 0.8633379
5 0.61951596
6 0.6507812
6 0.92076313
5 0.60313445
6 0.903519
6 0.93377024
6 0.7954593
6 0.8973837
5 0.64520574
6 0.6583886
4 0.888097
6 0.42748177
6 0.84407175
6 0.8902203
6 0.40357903
6 0.8891021
6 0.9402174
6 0.9022986
6 0.86397123
6 0.86464417
6 0.78863573
6 0.8217812
6 0.87239957
6 0.8892406
6 0.90924037
6 0.8307766
6 0.94251513
6 0.86976206
6 0.43667665
6 0.8504616
5 0.21617
6 0.8877349
6 0.45577267
6 0.910942
6 0.63723737
6 0.78286475
6 0.92617595
6 0.9361058
6 0.8183762
6 0.5331694
6 0.93570745
6 0.87371796
6 0.87287843
5 0.42278427
6 0.7499668
6 0.87546897
6 0.9426414
5 0.58745676
5 0.5822712
6 0.91438216
6 0.8578608
6 0.46282548
6 0.7320634
6 0.46145567
5 0.4073414
6 0.7020312
6 0.54570144
6 0.8133108
6 0.65988773
6 0.84869736
6 0.9206546
6 0.64655524
6 0.90438074
5 0.5618011
6 0.523291
5 0.4560528
6 0.54762316
6 0.8277531
6 0.8070675
5 

6 0.8242351
6 0.79877555
6 0.78348416
6 0.66128117
5 0.55559915
5 0.50638014
6 0.85906154
5 0.8751644
5 0.78318805
6 0.5062316
6 0.7905709
5 0.81678396
6 0.5306504
5 0.90237665
5 0.6160221
5 0.4229133
5 0.736923
5 0.8028936
5 0.92678815
6 0.964896
6 0.634644
6 0.8381894
6 0.8321993
6 0.82847095
6 0.6203744
6 0.68522847
6 0.66980976
6 0.8117033
5 0.8241001
5 0.65504223
6 0.5659689
6 0.80124813
6 0.55419046
6 0.6753993
5 0.7637988
5 0.83967274
5 0.7668921
6 0.6613912
6 0.56460047
5 0.6693992
5 0.80253524
5 0.39733526
5 0.78778285
6 0.46581966
5 0.7633609
6 0.8529045
6 0.55135316
5 0.84005535
5 0.8890199
5 0.74251884
5 0.63276
5 0.6249102
5 0.8457768
5 0.771997
6 0.596772
6 0.7856354
5 0.6932732
5 0.5393573
5 0.66910565
5 0.8656626
5 0.53905827
5 0.74805593
6 0.8216885
6 0.80951166
6 0.76352507
5 0.6259971
5 0.7561457
5 0.77352417
6 0.57614416
5 0.89030415
5 0.77562416
6 0.6423962
6 0.42735657
6 0.5024946
6 0.77133226
5 0.88656276
6 0.5598784
6 0.8242093
6 0.41092184
6 0.73398405
6 0.6438

6 0.6036596
6 0.5072116
6 0.6237746
6 0.6765774
4 0.3898093
5 0.3578144
4 0.49919057
6 0.65599036
6 0.54398984
5 0.8866165
6 0.88160217
6 0.9016658
6 0.81226665
6 0.49587944
6 0.5858234
6 0.76217693
5 0.7889058
6 0.45653045
6 0.57230633
5 0.59117603
6 0.5275324
6 0.8808919
5 0.75411576
6 0.88758343
6 0.5713752
6 0.694257
6 0.9112541
6 0.6693108
5 0.48435163
6 0.90234256
5 0.4579107
6 0.7099037
6 0.57382
6 0.89435863
4 0.4848557
6 0.8144352
5 0.89807945
6 0.6607948
5 0.80084586
5 0.47642964
4 0.580056
6 0.93907607
4 0.49180803
5 0.6190434
6 0.52320784
6 0.94460624
6 0.80957985
6 0.88523805
6 0.72917616
6 0.60454226
5 0.70220137
5 0.8274707
6 0.79052985
6 0.53105235
6 0.41259068
6 0.7160535
6 0.5868675
6 0.5074058
5 0.376306
5 0.5742194
5 0.9329458
5 0.77446884
6 0.4875936
5 0.62775177
5 0.7215349
5 0.8658729
5 0.874685
6 0.5494766
6 0.7903941
6 0.68767965
6 0.75028276
6 0.8300181
6 0.59082973
6 0.6343748
5 0.6341736
5 0.75197726
5 0.697669
6 0.6699495
6 0.67119753
5 0.8768738
6 0.674368

6 0.72230536
5 0.5067376
6 0.5584137
5 0.8087978
5 0.4353574
6 0.87179387
6 0.47726014
5 0.5378582
6 0.8140794
6 0.86757356
5 0.62311167
5 0.478533
6 0.7823982
5 0.7498435
5 0.48510286
6 0.91399044
6 0.71767765
5 0.7957799
5 0.8110321
6 0.61509323
5 0.7069657
5 0.46837434
6 0.41611537
6 0.3992956
6 0.8577173
5 0.64176977
5 0.72750556
5 0.5118678
5 0.40441278
5 0.46193558
5 0.86963123
6 0.5030363
6 0.36822665
5 0.8177217
5 0.36351573
5 0.6989448
6 0.3807289
5 0.6942612
6 0.3737466
5 0.59238505
6 0.38090533
6 0.4106552
6 0.73307985
6 0.37386227
5 0.44240072
6 0.7116894
5 0.73983365
6 0.8095942
6 0.42783812
5 0.78622365
5 0.8486901
6 0.74593025
5 0.42369822
6 0.8001479
6 0.6299178
6 0.39830393
5 0.75026184
6 0.37235278
5 0.4575304
6 0.8594712
5 0.6924754
5 0.5368727
6 0.6693916
6 0.7703019
6 0.5287896
5 0.5132898
6 0.78872776
6 0.40416214
6 0.4764065
5 0.7710515
5 0.5975298
6 0.8969617
6 0.45214656
6 0.51142734
5 0.5058441
6 0.77080905
6 0.4669656
5 0.44209734
6 0.33852348
5 0.6574947
5 0

4 0.41713172
4 0.9909863
4 0.6144134
4 0.4159496
4 0.95311165
4 0.72234637
6 0.3638125
4 0.5421524
5 0.53800595
6 0.6718492
6 0.8284688
6 0.67651916
6 0.76384586
4 0.63562435
6 0.48463896
4 0.6726483
4 0.99658316
6 0.6763053
4 0.618671
6 0.6495451
5 0.3647303
4 0.99856037
4 0.9383245
4 0.9226731
4 0.5550716
6 0.43218708
6 0.3440192
6 0.6534804
4 0.9923476
6 0.5852767
4 0.98154825
4 0.79788667
4 0.5569207
4 0.9882085
4 0.8916096
4 0.9792733
4 0.42859793
6 0.4008946
4 0.7157691
4 0.9129471
4 0.6420188
4 0.94052494
6 0.51495796
5 0.57776785
5 0.78553206
5 0.8959091
5 0.91277385
5 0.94757205
5 0.92743903
5 0.83986115
5 0.64952993
6 0.3891793
5 0.89083695
5 0.8061712
6 0.9343216
5 0.76023304
5 0.7524109
6 0.49227625
6 0.47214755
5 0.8585145
5 0.7686336
5 0.77469033
6 0.93615
5 0.6246577
5 0.92479616
5 0.73153836
5 0.43168223
5 0.7899668
6 0.65260524
5 0.8771638
6 0.8356371
6 0.7768252
5 0.44310713
5 0.8904284
6 0.49534464
6 0.45853388
6 0.801219
5 0.91045034
5 0.4895347
5 0.8810616
5 0.7740

...

9 0.9738279
9 0.95111674
9 0.9622352
9 0.96277803
9 0.9543612
9 0.97163683
9 0.9614142
9 0.9668109
9 0.94618845
9 0.95176905
9 0.980056
9 0.96319824
9 0.9674489
9 0.9659995
9 0.9687932
9 0.98159665
9 0.96227705
9 0.96999925
9 0.9684891
9 0.95796275
9 0.950194
9 0.9817152
9 0.9655817
9 0.9732423
9 0.98407745
9 0.98043174
9 0.9557126
9 0.9671864
9 0.99027914
9 0.94382894
9 0.9657328
9 0.9279463
9 0.95629746
9 0.9660609
9 0.97489953
9 0.9565321
9 0.9645796
9 0.9660609
9 0.9607193
9 0.97474426
9 0.9508364
9 0.97259504
9 0.96179837
9 0.9509244
9 0.97369486
9 0.97644037
9 0.9631739
9 0.9821567
9 0.9570087
9 0.9700763
9 0.9752349
9 0.9577791
9 0.9601474
9 0.983943
9 0.98312366
9 0.98460096
9 0.9662125
9 0.95808345
9 0.9543149
9 0.94793826
9 0.97420985
9 0.9353643
9 0.9747406
9 0.97138613
9 0.94588184
9 0.9609259
9 0.9540675
9 0.97636217
9 0.9599787
9 0.96572876
9 0.97767764
9 0.9810662
9 0.96153384
9 0.9876215
9 0.93916094
9 0.96710783
9 0.99193317
9 0.95827365
9 0.9671864
9 0.9833544
9 0.957

9 0.7845732
9 0.78249204
9 0.900017
9 0.73785937
9 0.92474806
9 0.87547547
9 0.79440033
9 0.8672262
9 0.74721533
9 0.7346705
9 0.80987513
9 0.75603807
9 0.7346705
9 0.7712457
9 0.7610316
9 0.82305986
9 0.76304734
9 0.76476794
9 0.7649305
9 0.7508538
9 0.80535823
9 0.68435717
9 0.8255519
9 0.7208481
9 0.8736952
9 0.75135726
9 0.64174825
9 0.79653853
9 0.7941543
9 0.77652353
9 0.7157128
9 0.7923482
9 0.84945285
9 0.9132059
9 0.82064545
9 0.84880143
9 0.8695398
9 0.8644395
9 0.8363395
9 0.80574614
9 0.79612345
9 0.8315802
9 0.8256843
9 0.83085734
9 0.744958
9 0.6332571
9 0.7018828
9 0.71041214
9 0.77612877
9 0.52179104
9 0.56178206
9 0.5399277
9 0.4837307
9 0.475541
9 0.4760643
9 0.6322302
9 0.7345607
9 0.77462095
9 0.81065106
9 0.6905575
9 0.8091637
9 0.7902703
9 0.8487782
9 0.7400169
9 0.8152973
9 0.8264099
9 0.9226146
9 0.9191727
9 0.94605947
9 0.97071695
9 0.97018373
9 0.9346308
9 0.9293109
9 0.92555207
9 0.9117298
9 0.95525676
9 0.91273564
9 0.9185019
9 0.9458163
9 0.90720606
9 0.921

9 0.8797869
9 0.86809254
9 0.8643718
9 0.8349255
9 0.99137294
9 0.9999888
9 0.70987445
9 0.7507769
9 0.8326898
9 0.45988148
9 0.73863363
9 0.95187026
9 0.98516256
9 0.9866064
9 0.9812516
9 0.9767007
9 0.96443576
9 0.9763716
9 0.99143815
9 0.98617744
9 0.99998736
9 0.9819406
9 0.9982607
9 0.99999833
9 0.9999713
9 0.9991937
9 0.99930215
9 0.99913424
9 0.999918
9 0.99993885
9 0.8619923
9 0.9875072
9 0.9990546
9 0.97011346
9 0.99373865
9 0.9892463
9 0.9932221
9 0.9925069
9 0.98881906
9 0.9962721
9 0.9973494
9 0.99799573
9 0.99696594
9 0.99321765
9 0.9989913
9 0.99344563
9 0.9952696
9 0.9858714
9 0.9898811
9 0.99658835
9 0.9960537
9 0.99422693
9 0.9643997
9 0.98888665
9 0.99377537
9 0.98897445
9 0.9924372
9 0.99235606
9 0.96794957
9 0.9833514
9 0.98453194
9 0.9977848
9 0.9881374
9 0.9437522
9 0.9907879
9 0.9844007
9 0.990947
9 0.9860881
9 0.9798727
9 0.9999883
9 0.96957153
9 0.97196376
9 0.95961624
9 0.9443083
9 0.9733006
9 0.9687382
9 0.86719096
9 0.8457487
9 0.89633733
9 0.862297
9 0.8647

9 0.47353986
9 0.46740472
9 0.5645241
9 0.510004
9 0.52921784
9 0.52072287
9 0.5132498
9 0.5263926
9 0.58400965
9 0.6351374
9 0.58092916
9 0.59960496
9 0.6511713
6 0.9624162
6 0.6015395
6 0.51254797
9 0.57896763
9 0.76389587
9 0.9307368
9 0.9090749
9 0.91248214
9 0.97723037
9 0.9936714
9 0.9945797
9 0.9843076
9 0.9999565
9 0.9999956
9 0.99998975
9 0.9999912
9 0.9987526
9 0.99999654
9 0.9966186
9 0.99984646
9 0.7012611
9 0.99994373
9 0.9992137
9 0.9989421
9 0.9984499
9 0.9999442
9 0.9999126
9 0.9994313
9 0.99995816
9 0.998329
9 0.9993405
9 0.99914205
9 0.9761054
9 0.9982823
9 0.99248344
9 0.9910651
9 0.97918314
9 0.9812664
9 0.9982657
9 0.97735775
9 0.9764147
9 0.9924217
9 0.967811
9 0.9745588
9 0.9690186
9 0.97856754
9 0.9865291
9 0.96789503
9 0.97262675
9 0.9613366
9 0.9617744
9 0.96543825
9 0.9695226
9 0.9708311
9 0.9909161
9 0.9862298
9 0.96341085
9 0.961404
9 0.9469287
9 0.9649604
9 0.95362836
9 0.9516693
9 0.9660204
9 0.98938274
9 0.9940752
9 0.9738225
9 0.98703617
9 0.9897035
9 0

9 0.98129463
9 0.9794109
9 0.97924656
9 0.9878197
9 0.9801297
9 0.9869223
9 0.9867025
9 0.9778246
9 0.9787145
9 0.9813229
9 0.96954894
9 0.95369285
9 0.9346308
9 0.93775964
9 0.8559838
9 0.88975185
9 0.82409054
9 0.86728317
9 0.99875164
6 0.58743966
6 0.72052693
6 0.62501717
9 0.6925082
9 0.85165775
9 0.85976785
9 0.92984957
9 0.95843613
9 0.93032545
9 0.9999969
9 0.99999774
9 0.9775759
9 0.98098904
9 0.9999999
9 0.99999905
9 0.99998665
9 0.9696932
9 0.99998283
9 0.99834967
9 0.99999404
9 0.9999945
9 0.9999825
9 0.9999931
9 0.9971802
9 0.99998915
9 0.99999714
9 0.9267689
9 0.9580349
9 0.9467036
9 0.96007615
9 0.9879913
9 0.9815144
9 0.99500024
9 0.985471
9 0.99027
9 0.9909759
9 0.9826398
9 0.988052
9 0.9861335
9 0.9922805
9 0.9892174
9 0.99999404
9 0.9999963
9 0.99019027
9 0.99822146
6 0.67055225
9 0.99509346
9 0.99964666
9 0.5203545
9 0.9992859
9 0.9877297
9 0.9580932
9 0.9792307
9 0.9915542
9 0.99729544
9 0.9912168
9 0.97276014
9 0.9953791
9 0.95411134
9 0.9922559
9 0.9411164
9 0.994